# Zagadki

## Wstęp
Zagadki od wieków fascynują ludzi, pobudzając ich umysły do kreatywnego i logicznego myślenia. Od prostych łamigłówek po głębokie zagadki filozoficzne, stanowią one nie tylko formę rozrywki, ale również sztukę rozumienia języka i logicznego wnioskowania. W tym zadaniu będziesz rozwiązywał/a zagadki, polegające na odgadnięciu słowa na podstawie opisu. Wszystkie zagadki wymyślił ChatGPT (ale nie powiemy, która dokładnie wersja i w jaki sposób zachęcona), stąd niektóre mogą być trochę dziwne... Szacujemy, że ludzie są w stanie rozwiązać poprawnie trochę ponad 60% z nich. A jak dobry będzie Twój program?

## Zadanie
Napisz funckję `answer_riddle` która rozwiąże podaną na wejściu zagadkę. Rozwiązaniem jest zawsze jedno słowo. Przykładowe zagadki:

- **zagadka:** kobieta podróżująca środkiem transportu, np. samolotem, pociągiem, statkiem <br>
  **odpowiedź:** pasażerka
- **zagadka:** emocjonalne uczucie łączące dwie osoby, oparte na zaufaniu, szacunku, trosce i oddaniu<br>
  **odpowiedź:** miłość


Naszym kryterium będzie `odwrócona średnia harmoniczna` ([Mean Reciprocal Rank](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)), która działa w następujący sposób: <br>
Jeżeli na zwróconej przez Twoją funkcję liście znajdzie się prawidłowa odpowiedź, otrzymasz wówczas punkty: dokładnie $\frac{1}{k}$ punktów, gdzie $k$ jest pozycją słowa na liście. W szczególności, jeżeli Twój program zgadnie słowo (czyli umieści je na pierwszej pozycji), to otrzymasz 1 punkt. Ostatecznym kryterium jest uśredniona liczba punktów ze wszystkich zagadek.

Powyższe kryterium jest zaimplementowane poniżej przez nas.

## Ograniczenia
- Twoje finalne rozwiązanie będzie testowane w środowisku **bez** GPU.
- Twoja funkcja powinna działać na tyle szybko, aby program był w stanie udzielić odpowiedzi na 100 zagadek w maksymalnie 2 minuty bez użycia GPU.

## Dane
Dane dostępne dla Ciebie w tym zadaniu to:
* `zagadki_do_testow_clean.txt` - około 2000 przykładowych zagadek
* `plwiktionary_definitions_clean.txt` -  plik z definicjami słów wziętymi z [pl.wiktionary.org](https://pl.wiktionary.org/wiki/pl). Z wszystkich definicji z pl.wiktionary.org wzięliśmy definicje 8094 najbardziej popularnych rzeczowników (częstości liczone zgodnie z korpusem https://2018.poleval.pl/index.php/tasks#task3). Uwaga: poprawne rozwiązanie każdej zagadki znajduje się w tym pliku!

* `superbazy_clean.txt` - formy bazowe polskich słów, przygotowane na podstawie projektu https://github.com/morfologik/polimorfologik

* Wektory osadzeń słów bazowych, wytrenowane modelem Word2Vec z biblioteki Gensim, na korpusie PolEval 2018 Task3

## Uwagi i wskazówki
- Dla każdej zagadki, Twoja funkcja powinna zwrócić listę słów (co najwyżej 20), w kolejności od najbardziej (wg Twojego programu) prawdopodobnej odpowiedzi na zagadkę, do najmniej.
- Twoje rozwiazanie bedzie testowane bez dostepu do internetu

## Pliki Zgłoszeniowe
Tylko ten notebook.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` będziesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających. 

Za to zadanie możesz zdobyć pomiędzy 0 i 1.5 punktu. Zdobędziesz 0 punktów jeśli wartość kryterium `mean reciprocal rank` na zbiorze testowym wyniesie poniżej 0.02, a 1.5 punktu jeśli wyniesie powyżej 0.3. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością kryterium.

# Kod startowy

In [19]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

In [14]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
import nltk
from nltk.tokenize import word_tokenize as tokenize
from collections import defaultdict as dd
import math
from gensim.models import Word2Vec
import gdown
import random
import os
from tqdm import tqdm
import torch
import gc
import numpy as np
from transformers import AutoTokenizer, AutoModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
from transformers import logging
from pandas import Series

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.set_verbosity_error()
gc.collect()
torch.cuda.empty_cache()

## Ładowanie danych

In [15]:
path_to_data = './data/'

bases = {}
# Dictionary mapping words to their base words
all_word_definitions = dd(list)
# Dictionary containing all base words inverse document frequency
base_idf = dd(int)

In [17]:
def get_word_base(word):
    global bases
    word = word.lower()
    ret = bases.get(word)
    if ret:
        return ret
    return word

In [20]:
if not FINAL_EVALUATION_MODE:
    if not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.syn1neg.npy") \
        or not os.path.exists(f"{path_to_data}/zagadki/w2v_polish_lemmas.model.wv.vectors.npy"):
            gdown.download_folder(url="https://drive.google.com/drive/folders/1P72og_ORfL3Ojf27n-g06DT0ENduPy8C?usp=sharing", output=f"./{path_to_data}")
    nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/patryk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### tworzenie bases

In [21]:
for x in open(f'{path_to_data}/zagadki/superbazy_clean.txt'):
    word,base = x.lower().split()
    bases[word] = base

#### ladowanie modeli

In [22]:
model_word2vec = Word2Vec.load(f'{path_to_data}/zagadki/w2v_polish_lemmas.model')

model_name_pauga = 'flax-community/papuGaPT2'
tokenizer_papuga = AutoTokenizer.from_pretrained(model_name_pauga)
model_papuga = AutoModelForCausalLM.from_pretrained(model_name_pauga).to(device)
emb_papuga = model_papuga.transformer.wte.weight.detach().cpu().numpy()
del model_papuga

model_name_polka = 'eryk-mazus/polka-1.1b'
tokenizer_polka = AutoTokenizer.from_pretrained(model_name_polka)
model_polka = AutoModelForCausalLM.from_pretrained(model_name_polka).to(device)

model_name_herbert = "allegro/herbert-base-cased"
tokenizer_herbert = AutoTokenizer.from_pretrained(model_name_herbert)
model_herbert = AutoModel.from_pretrained(model_name_herbert).to(device)

In [23]:
gc.collect()

8

#### tworzenie answers i queries

In [24]:
answers = []
queries = []

with open(f'{path_to_data}/zagadki/zagadki_do_testow_clean.txt') as file:
    for line in file:
        line = line.replace(';;', '').split()                  
        answers.append(line[0])
        queries.append(' '.join(line[1:]))

In [25]:
queries[0], answers[0]

('rękopiśmienny tekst lub dokument, niepublikowany drukiem.', 'manuskrypt')

#### tworzenie all_word_definitions, base_idf, query, answer

In [26]:
punctuation = {'!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '–', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '``', "''"}

for x in open(f'{path_to_data}/zagadki/plwiktionary_definitions_clean.txt'):
    word, definition = x.split('###')
    L = word.split()
    if len(L) == 1:
        word = L[0]
        definition = definition.replace('\n', '')
        definition = definition.replace('|', ' ')
        definition = ''.join(c for c in definition.lower() if c not in punctuation)
        if len(definition) == 0: continue
        all_word_definitions[word].append(definition)
        for word in set(definition.split()):
            base_idf[get_word_base(word)] += 1

for query, answer in tqdm(zip(queries[293:], answers[293:]), total=len(queries[293:])):
    all_word_definitions[answer].append(''.join(c for c in query if c not in punctuation))
    for word in set(query.split()):
        base_idf[get_word_base(word)] += 1

for base in base_idf:
    base_idf[base] = -math.log(base_idf[base] / len(all_word_definitions))

100%|██████████| 1700/1700 [00:00<00:00, 38331.74it/s]


In [27]:
Series(base_idf).describe()

count    21982.000000
mean         8.175645
std          1.116514
min          0.611736
25%          7.611842
50%          8.998137
75%          8.998137
max          8.998137
dtype: float64

In [28]:
for w, v in base_idf.items():
    if v < 2.5: print(w, v)

wikipedia 1.2066139415495751
w 0.6117358595340934
on 2.4216671916520833
o 2.1427279620903796
lub_lubić 0.7472556159996584
od 2.4849066497880004
z 1.1955186972576357
do 1.2049624135111021
coś 1.680260562073812
jakiś 2.4470564256569025
osoba 1.9824243402130781
przen 2.2924976658403047
i 1.2540001330723172
który 1.5336269260637798
pot 1.9743778059618642
się 1.4333797477945782
na 1.2830131570682022
przez 2.492352700572079
ten 2.4599969369326375


In [29]:
all_word_definitions['morze']

[' geogr hydrol duży zbiornik słonej wody mający połączenie z oceanem wikipedia',
 ' przen ogromna ilość czegoś',
 'duże słone zbiornik wodny połączone z oceanem zazwyczaj otoczone lądem']

## Kod z kryteriami oceniającymi

In [30]:
def mean_reciprocal_rank(real_answers, computed_answers, K=20):
    positions = []

    for real_answer, computed_answer in zip(real_answers, computed_answers):
        if real_answer in computed_answer[:K]:
            pos = computed_answer.index(real_answer) + 1
            positions.append(1/pos)
    
    mrr = sum(positions) / len(real_answers)
    print ('Mean Reciprocal Rank =', mrr)
    
    return mrr

# Twoje rozwiązanie

To jest jedyna sekcja, w której musisz coś zrobić.

In [58]:
def cos(a, b, epsilon=1e-10):
    return a.dot(b) / ((a.dot(a) * b.dot(b)) ** 0.5 + epsilon)

### PAPUGA & IDF

In [64]:
def decoder_embedding(word):
    token_ids = tokenizer_papuga.encode(' ' + word)
    # emb_papuga[token_ids[0]] *= 3.0
    # whole_word_embbeding = np.sum(np.stack([emb_papuga[id] for id in token_ids]), axis=0)
    whole_word_embbeding = np.sum(np.stack([emb_papuga[id] for id in token_ids]), axis=0) / len(token_ids)
    return whole_word_embbeding

In [66]:
all_word_definitions_papuga_embeddings = dd(list)

for word in tqdm(all_word_definitions, total=len(all_word_definitions)):
    for definition in all_word_definitions[word]:
        definition = list(definition)
        first_w_in_definition = definition[0]
        base = get_word_base(first_w_in_definition)
        res_embbeding = decoder_embedding(first_w_in_definition) * -base_idf[get_word_base(base)]
        # res_embbeding = decoder_embedding(first_w_in_definition)
        for w in definition[1:]:
            base = get_word_base(w)
            res_embbeding += decoder_embedding(w) * -base_idf[base]
            # res_embbeding += decoder_embedding(w)
        all_word_definitions_papuga_embeddings[word].append(res_embbeding)

100%|██████████| 8090/8090 [02:11<00:00, 61.64it/s] 


In [34]:
def papuga_idf_answer_riddle_word_score(riddle, K):
    # papuga & idf & earest neighbor word2vec
    riddle = list(filter(lambda x: x not in punctuation, riddle))
    res_embbeding = decoder_embedding(riddle[0]) * -base_idf[get_word_base(riddle[0])]
    # res_embbeding = decoder_embedding(riddle[0])
    for word in riddle[1:]:
        res_embbeding += decoder_embedding(word) * -base_idf[get_word_base(word)]
        # res_embbeding += decoder_embedding(word)
    
    closest_words = []
    for word, embeddings in all_word_definitions_papuga_embeddings.items():
        for emb in embeddings:
            # distance = np.linalg.norm(res_embbeding - emb)
            distance = cos(res_embbeding, emb)
            closest_words.append((distance, word))
    
    closest_words = sorted(closest_words, key=lambda x: x[0])
    return closest_words[-K:]

### HERTBER & IDF ( aktualnie bez idf )

In [35]:
def encoder_embedding(L):
    txt = ' '.join(L)
    input_ids = tokenizer_herbert(txt, return_tensors='pt')['input_ids'].to(device)
    output = model_herbert(input_ids=input_ids)
    return output.last_hidden_state.detach().cpu().numpy()[0,0,:]

In [36]:
all_word_definitions_herbert_embeddings = dd(list)

for word in tqdm(all_word_definitions):
    for definition in all_word_definitions[word]:
        def_embbeding = encoder_embedding(definition)
        all_word_definitions_herbert_embeddings[word].append(def_embbeding)

100%|██████████| 8088/8088 [06:01<00:00, 22.38it/s]


In [37]:
def herbert_idf_answer_riddle_word_score(riddle, K):
    # riddle = list(filter(lambda x: x not in punctuation and base_idf[x] > 2.5, riddle))
    riddle = list(filter(lambda x: x not in punctuation, riddle))
    riddle_embbeding = encoder_embedding(riddle)

    closest_words = []
    for word, embeddings in all_word_definitions_herbert_embeddings.items():
        # idf = base_idf[get_word_base(word)]
        for emb in embeddings:
            # distance = np.linalg.norm(res_embbeding - emb)
            distance = cos(riddle_embbeding, emb)
            # distance = idf * cos(riddle_embbeding, emb)
            closest_words.append((distance, word))
    
    closest_words = sorted(closest_words, key=lambda x: x[0])
    return [ x[1] for x in closest_words[-K:]]
    # return closest_words[-K:]

### WORD2VEC & IDF

In [38]:
from numpy import dot
from numpy.linalg import norm
def cos_np(a, b):
    if a.shape != b.shape:
        print(a.shape, b.shape)
    assert a.shape == b.shape
    return dot(a, b)/(norm(a)*norm(b) + np.finfo(float).eps)

In [39]:
def word2vec_embedding(word):
    try:
        return model_word2vec.wv[word]
    except:
        return np.zeros(200)

In [40]:
all_word_definitions_wrd2vec_embeddings = dd(list)

for word in tqdm(all_word_definitions):
    for definition in all_word_definitions[word]:
        definition_tekenized = tokenize(definition)
        def_embbeding = np.sum([word2vec_embedding(get_word_base(w)) for w in definition_tekenized if w not in punctuation and base_idf[get_word_base(w)] > 1.9], axis=0)
        if not isinstance(def_embbeding, np.ndarray): continue
        all_word_definitions_wrd2vec_embeddings[word].append(def_embbeding)
        

100%|██████████| 8088/8088 [00:02<00:00, 3211.51it/s]


In [41]:
def word2vec_idf_answer_riddle_word_score(riddle, K):
    riddle_embbeding = np.sum([word2vec_embedding(get_word_base(w)) for w in riddle if base_idf[get_word_base(w)] > 1.9], axis=0)

    closest_words = []
    for word, embeddings in all_word_definitions_wrd2vec_embeddings.items():
        for emb in embeddings:
            distance = cos_np(riddle_embbeding, emb)
            closest_words.append((distance, word))
    
    closest_words = sorted(closest_words, key=lambda x: x[0])
    return closest_words[-K:]

### POLKA & filtrowanie generacji

In [73]:
prompt = """Przeanalizuj podane zdanie i odpowiedz na nie jednym słowem, które najlepiej pasuje do treści. Twoja odpowiedź powinna zawierać tylko jeden wyraz.

Przykłady:

Zagadka: mały owad produkujący miód.
Odpowiedź: pszczoła

Zagadka: drapieżny ssak, uważany za króla dżungli.
Odpowiedź: lew

Zagadka: osoba, która pisze książki.
Odpowiedź: pisarz

Do rozwiązania:
Zagadka: [[zagadka]]
Odpowiedź:"""

In [74]:
def polka_answer_riddle(riddle, K):
    prompt_filled = prompt.replace('[[zagadka]]', riddle)
    input_ids = tokenizer_polka(prompt_filled, return_tensors='pt').input_ids.to(device)
    output = model_polka.generate(input_ids, max_length=300, num_return_sequences=K, do_sample=True, top_k=K)
    answers = [tokenizer_polka.decode(output[i], skip_special_tokens=True).split('Odpowiedź:')[-1].strip().split()[0] for i in range(K)]
    answers = list(filter(lambda x: x not in all_word_definitions.keys(), answers))
    return answers

### ML approach

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

X = []
y = []
for query, answer in zip(queries[293:], answers[293:]):
    query_embedding = decoder_embedding(query) # PAPUGA
    # query_embedding = encoder_embedding(query) # HERBERT
    num_definitions = len(all_word_definitions[answer])
    negative_samples = []

    while len(negative_samples) < num_definitions * 2:
        # tutja jeszcze powinien byc set w licie aby bylo bez powtorzen
        negative_samples = random.sample(list(all_word_definitions_papuga_embeddings.keys()), num_definitions * 2)
        # negative_samples = random.sample(list(all_word_definitions_herbert_embeddings.keys()), num_definitions * 2)
    
    for emb in all_word_definitions_papuga_embeddings[answer]: # PAPUGA
    # for emb in all_word_definitions_herbert_embeddings[answer]: # HERBERT
        combined_embedding = np.concatenate((query_embedding, emb))
        for i in range(2): # żeby było tyle samo positive samples i negative samples
            X.append(combined_embedding)
            y.append(1)  # correct answer

    for word in negative_samples:
        for emb in all_word_definitions_papuga_embeddings[word]: # PAPUGA
        # for emb in all_word_definitions_herbert_embeddings[word]: # HERBERT
            combined_embedding = np.concatenate((query_embedding, emb))
            X.append(combined_embedding)
            y.append(0)  # incorrect answer

print(1)

# X_train, X_test, y_train, y_test = X[:1700], X[1700:], y[:1700], y[1700:]
X_train = np.array(X)
y_train = np.array(y)

X = []
y = []
for query, answer in zip(queries[:293], answers[:293]):
    query_embedding = decoder_embedding(query) # PAPUGA
    # query_embedding = encoder_embedding(query) # HERBERT
    
    for emb in all_word_definitions_papuga_embeddings[answer]: # PAPUGA
    # for emb in all_word_definitions_herbert_embeddings[answer]: # HERBERT
        combined_embedding = np.concatenate((query_embedding, emb))
        X.append(combined_embedding)
        y.append(1)  # correct answer

print(2)

X_test = np.array(X)
y_test = np.array(y)

model = LogisticRegression(max_iter=10000, C=0.1)  # Adding L2 regularization with C=0.1
model.fit(X_train, y_train)

print(3)

train_score = model.score(X_train, y_train)

print(4)

val_score = model.score(X_test, y_test)

print(f"Training accuracy: {train_score}")
print(f"Test accuracy: {val_score}")


1
2
3
4
Training accuracy: 0.7633079686090257
Test accuracy: 0.0


In [51]:
def ml_answer_riddle(riddle, K):
    riddle_embedding = encoder_embedding(riddle)
    scores = []
    for word, embeddings in all_word_definitions_herbert_embeddings.items():
        for emb in embeddings:
            combined_embedding = np.concatenate((riddle_embedding, emb))
            score = model.predict_proba([combined_embedding])[0][1]  # probability of being the correct answer
            scores.append((score, word))
    
    scores = sorted(scores, key=lambda x: x[0], reverse=True)
    return [word for _, word in scores[:K]]

In [69]:
queries[0], answers[0]

('rękopiśmienny tekst lub dokument, niepublikowany drukiem.', 'manuskrypt')

In [70]:
ml_answer_riddle(queries[0], 20)

['lampa',
 'łoże',
 'tur',
 'as',
 'por',
 'contra',
 'om',
 'om',
 'pan',
 'gros',
 'zapowiedź',
 'limit',
 'modus',
 'ciemny',
 'cola',
 'focus',
 'kres',
 'schemat',
 'ograniczenie',
 'agent']

#### puting all togethr

In [ ]:
def answer_riddle():
    pass

# Ewaluacja

Poniższy kod będzie służył ewaluacji rozwiązania. Po wysłaniu rozwiązania do nas zostanie wykonana funkcja `evaluate_algorithm(score_function, queries, answers, K)`, t.j. prawie identyczny kod jak niżej będzie się uruchamiał na katalogu danych `test_data` dostępnym tylko dla sprawdzających zadania.

Upewnij się przed wysłaniem, że cały notebook wykonuje się od początku do końca bez błędów i bez ingerencji użytkownika po wykonaniu polecenia `Run All`.

In [47]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
def evaluate_algorithm(score_function, queries, answers, K):
    computed_answers = []
    for query in tqdm(queries, desc="queries answered"):
        computed_answers.append(score_function(query, K=K))
    score = mean_reciprocal_rank(answers, computed_answers, K=K)
    
    return score

In [48]:
gc.collect()

52

In [75]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################
if not FINAL_EVALUATION_MODE:
    PART_OF_DATA = 293
    K = 20
    valid_queries = queries[:PART_OF_DATA]
    valid_answers = answers[:PART_OF_DATA]
    score = evaluate_algorithm(polka_answer_riddle, valid_queries, valid_answers, K=K)
    print(f"Score: {score}")

queries answered:   1%|▏         | 4/293 [00:11<13:14,  2.75s/it]


KeyboardInterrupt: 

 herbert \
queries answered: 100%|██████████| 293/293 [01:09<00:00,  4.19it/s] \
Mean Reciprocal Rank = 0.002384157962328695 \
Score: 0.002384157962328695

ml \
queries answered: 100%|██████████| 293/293 [17:14<00:00,  3.53s/it] \
Mean Reciprocal Rank = 0.0005281976271737364 \
Score: 0.0005281976271737364

word2vec idf \
queries answered: 100%|██████████| 293/293 [02:03<00:00,  2.38it/s] \
Mean Reciprocal Rank = 0.0 \
Score: 0.0

papuga suma ebedow slow wazona  -idf \
queries answered: 100%|██████████| 293/293 [01:01<00:00,  4.74it/s] \
Mean Reciprocal Rank = 0.020161534304620136 \
Score: 0.020161534304620136

